# Homework 1: How to get XOR eyes

*Expected due date:* April 26, 2021 (Monday)

This homework is designed to get you started in implementing some cryptographic algorithms in Python.
This should be easy enough for you to do solo, but it doesn't hurt if you want to work with a partner.
These exercises are important in the sense that they serve as stepping stones to future attacks, and 
most likely you'll need your code for these for the final contest.
If you can finish this homework, then you'll very likely do well in the final contest!

Much of classical crypto operates on alphabetic strings (strings only containing letters) while modern 
crypto deals excusively with binary strings.
Despite that crucial difference, we may see some familiar parallels between historical ciphers like 
Caesar or Vigenère and relatively modern ones like the XOR cipher and the one-time pad.

This homework has 32 points in total, but will be divided by 30 to get the final percentage. Final percentages are capped at 100%.

If any, please direct all your questions and clarifications about this homework in the `#hw1-help` channel on 
the Discord server.

## Dealing with binary data in Python

Python supports *byte literals* (I just call them "bytestrings" for convenience), string-like sequences that are prefixed by `b`.

In [1]:
b'This is a byte literal'

b'This is a byte literal'

In [2]:
'Meanwhile this is a string literal'

'Meanwhile this is a string literal'

A bytestring is not the same as its string counterpart...

In [3]:
'some_string' == b'some_string'

False

...because they have different types.

In [4]:
print(type('some_string'))
print(type(b'some_string'))

<class 'str'>
<class 'bytes'>


The usual string operations work with bytestrings.

In [5]:
test = b'this is a long bytestring'
print(test[3:10])   # slicing
print(test[4])      # get the 4th byte (returns an int)

b's is a '
32


In [6]:
another = b' and anotha one!'
print(test + another)   # concatenating bytestrings
print(another * 10)     # n-fold concatenation

b'this is a long bytestring and anotha one!'
b' and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one!'


We can use the following libraries to convert between encodings (hex and Base64).

In [7]:
from binascii import hexlify, unhexlify

In [8]:
from base64 import b64encode, b64decode

In [9]:
hexlify(b'this is gonna be hex soon')

b'7468697320697320676f6e6e612062652068657820736f6f6e'

In [10]:
unhexlify(b'7468697320697320686578206e6f206d6f7265')

b'this is hex no more'

In [11]:
b64encode(b'Base64 this thing!')

b'QmFzZTY0IHRoaXMgdGhpbmch'

In [12]:
b64decode(b'YmFzZTY0IGlzIG5vdCBlbmNyeXB0aW9uIQ==')

b'base64 is not encryption!'

## Some reminders

You are not allowed to use additional libraries (even within the Python standard library) other than those explicitly used here, though you may implement additional functions of your own.

**Very important:** Always work with raw bytes, never with encoded strings.

## Hex to Base64 [2 pts]

Write a function called `hex_to_b64` to convert a hex-encoded bytestring into Base64. For example, the string
```
48656c7021204920676f7420584f52206579657320616e642069742773206b696c6c696e67206d65
```
should output
```
SGVscCEgSSBnb3QgWE9SIGV5ZXMgYW5kIGl0J3Mga2lsbGluZyBtZQ==
```

In [13]:
def hex_to_b64(h):
    return b64encode(unhexlify(h))

In [14]:
# if your function works properly, no error should appear when running this line
assert hex_to_b64(b'48656c7021204920676f7420584f52206579657320616e642069742773206b696c6c696e67206d65') == b'SGVscCEgSSBnb3QgWE9SIGV5ZXMgYW5kIGl0J3Mga2lsbGluZyBtZQ=='

## XOR'ing two bytestrings [4 pts]

Write a function called `xor_bytes` that takes two bytestrings of equal length and outputs their XOR. For example, if I have a bytestring
```
49207374696c6c206861766520584f522065796573206e6f772077686174
```
after hex-decoding it, and when I XOR it against
```
3a4f1e11060209001b04180100302a3e5045141c5345170a040016292955
```
it should output
```
736f6d656f6e652073656e642068656c70206d7920657965732061414821
```

In [15]:
def xor_bytes(a, b):
    assert len(a) == len(b), 'Both bytestrings must be equal in length'
    byte = ""
    for i in range(len(a)):
        byte = byte + chr(a[i] ^ b[i])
    byte = str.encode(byte)
    return byte

In [16]:
# if your function works properly, no error should appear when running this line
assert hexlify(xor_bytes(unhexlify(b'49207374696c6c206861766520584f522065796573206e6f772077686174'), unhexlify(b'3a4f1e11060209001b04180100302a3e5045141c5345170a040016292955'))) == b'736f6d656f6e652073656e642068656c70206d7920657965732061414821'

## Single-byte XOR cipher [6 pts]

You could say that this is just the Caesar cipher but it works over binary strings instead. Anyways, I have encrypted the hex-encoded bytestring
```
264f0300190a4f0d06084f1c0a02061f1d06020a1c4f0e010b4f264f0c0e0101001b4f03060a4e
```
by XOR'ing it against a single character. Find out what key I used, and decrypt the message.

*Side quest:* You can do this by hand (since there are only 256 possible keys to choose from), but don't!
**Write code to do this for you.**
Have some way of "scoring" a piece of English plaintext, like using character frequency.
So you can try each possible key and output the one with the best score.

Write a function called `try_decrypt_xor` that does just that, i.e., it takes a bytestring and outputs the key and the decrypted message (and optionally the score). And yes, you may implement your own functions as well.

In [17]:
# taken from http://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html
# feel free to use this!
FREQ_TABLE = {'e': 12.02, 't': 9.10, 'a': 8.12, 'o': 7.68, 'i': 7.31, 'n': 6.95, 's': 6.28, 'r': 6.02, 'h': 5.92, 'd': 4.32, 'l': 3.98, 'u': 2.88, 'c': 2.71, 'm': 2.61, 'f': 2.30, 'y': 2.11, 'w': 2.09, 'g': 2.03, 'p': 1.82, 'b': 1.49, 'v': 1.11, 'k': 0.69, 'x': 0.17, 'q': 0.11, 'j': 0.10, 'z': 0.07, ' ': 19.18}

In [18]:
def try_decrypt_xor(c):
    b64 = c
    key = '\x00'
    message = b''
    score = 9999999999999999
    for l in ['\x00', '\x01', '\x02', '\x03', '\x04', '\x05', '\x06', '\x07', '\x08', '\t', '\n', '\x0b', '\x0c', '\r', '\x0e', '\x0f', '\x10', '\x11', '\x12', '\x13', '\x14', '\x15', '\x16', '\x17', '\x18', '\x19', '\x1a', '\x1b', '\x1c', '\x1d', '\x1e', '\x1f', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f']:
        bxor = xor_bytes(b64, str.encode(l*len(b64)))
#         print(bxor)
#         print("")
#         print("")
#         print("")
        cur_score = 0
        for i in set(bxor):
            count = 0
            for value in bxor:
                if value == i:
                    count += 1
            try:
                letter = chr(i).lower()
                add = FREQ_TABLE[letter] - count
            except:
                add = 30
            cur_score += add
        if cur_score < score:
            key = l
            message = bxor
            score = cur_score
    return [key, message, score]

In [19]:
inp = b'264f0300190a4f0d06084f1c0a02061f1d06020a1c4f0e010b4f264f0c0e0101001b4f03060a4e'
inp = unhexlify(inp)
print(try_decrypt_xor(inp))

['o', b'I love big semiprimes and I cannot lie!', 102.03999999999999]


## Detect single-character XOR [6 pts]

I have a [file](http://penoy.admu.edu.ph/~guadalupe154884/classes/csci184.03/files/hw1/xor_strings.txt) that contains 420 64-character hex-encoded strings, and **only one** of them has been encrypted by single-character XOR. 

Find that string.

*Hint:* Your code from the previous item should help.

In [20]:
fin = open('xor_strings.txt', 'r')

In [21]:
strings = fin.readlines()

In [22]:
m = 9999999999999999
secret = b''
encrypted = ""
for s in strings:
    i = s[0:64]
    i = unhexlify(i)
    out = try_decrypt_xor(i)
    if out[2] < m:
        m = out[2]
        encrypted = s
        secret = out[1]
print("Encrypted: " + encrypted)
print("Unencrypted: ")
print(secret)

Encrypted: 761f59504a515b1f4b575a1f575e461f56511f5e1f515a5a5b535a4c4b5e5c54

Unencrypted: 
b'I found the hay in a needlestack'


## Implement repeating-key XOR [4 pts]

Here are the first two lines of what people consider the most important piece of English literature:
```
I said, certified freak
Seven days a week
```
Encrypt it using repeating-key XOR, with the key "`WAP`".

The way repeating-key XOR works is that the first byte of plaintext will be XOR'd against `W`, the next `A`, the next `P`, then `W` again for the 4th byte, and so on. In effect, it will look something like this:
```
Plaintext:   I   s a i d ,   c e ...
Key:         W A P W A P W A P W...
```
If this reminds you of the Vigenère cipher, you're absolutely right! This *is* just Vigenère, but we're working over binary strings instead.

Write a function called `xor_repeating` that takes two bytestrings corresponding to the plaintext and the key, and encrypts it using repeating-key XOR. Using the plaintext and key above, your function should output:
```
1e61233628347b61333233243e2739322570313335362a5a042426322f70332029246131773635322a
```
when hex-encoded.

In [23]:
def xor_repeating(m, key):
    key = key*len(m)
    key = key[0:len(m)]
    return xor_bytes(m, key)

In [24]:
# if your function works properly, no error should appear when running this line
assert hexlify(xor_repeating(b'I said, certified freak\nSeven days a week', b'WAP')) == b'1e61233628347b61333233243e2739322570313335362a5a042426322f70332029246131773635322a'

## Break repeating-key XOR [10 pts]

I have [another file here](http://penoy.admu.edu.ph/~guadalupe154884/classes/csci184.03/files/hw1/encrypted.txt).
It has been Base64-encoded after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let $\lambda$ be the guessed key length. Try values from $\lambda = 2$ up to $\lambda = 40$.

2. Write a function called `hamming_dist` that computes the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) between two bytestrings of equal length.
The Hamming distance is just the number of differing bits.
For example, the Hamming distance between `twelve plus one` and `ElEveN pLUs twO` is $30$. **Please make sure your function works correctly first before proceeding.**

3. For each $\lambda$, take the first $\lambda$ bytes and the second $\lambda$ bytes, and find the Hamming distance between them. 
Divide the result by $\lambda$ to normalize it.

4. The $\lambda$ with the smallest normalized Hamming distance is *probably* the key. But just to make sure, you could proceed with the smallest 2-3 $\lambda$ values, or you could take 4 $\lambda$ blocks instead of 2 and average their distances. Just a suggestion...

5. Now that you probably know $\lambda$, split the ciphertext into blocks of length $\lambda$.

6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on. For example, suppose we have the ciphertext split into blocks of length 5:
```
QWERT YUIOP ASDFG HJKLZ
```
then transposing these blocks will yield five blocks of length 4:
```
QYAH WUSJ EIDK ROFL TPGZ
```

7. Solve each block as if it was single-character XOR. You have code to do this, so go use that.

8. For each block, the single-byte XOR key that has the best score is the repeating-key XOR key byte for that block. All you need to do is to put them together and you have the key!

As before, you may implement additional functions of your own.

In [25]:
def hamming_dist(a, b):
    xor = xor_bytes(a, b)
    count = 0
    for b in xor:
        count += bin(b).count("1")
    return count

In [26]:
# if your function works properly, no error should appear when running this line
assert hamming_dist(b'twelve plus one', b'ElEveN pLUs twO') == 30

In [27]:
fin = open('encrypted.txt', 'rb')

In [28]:
b64str = fin.read()

In [29]:
s = b64decode(b64str)

In [30]:
quotients = []
for lam in range(2,41):
    dis = hamming_dist(s[:lam], s[lam:lam*2])
    dis2 = hamming_dist(s[lam*2:lam*3], s[lam*3:lam*4])
    dis3 = hamming_dist(s[lam*4:lam*5], s[lam*5:lam*6])
    dis4 = hamming_dist(s[lam*6:lam*7], s[lam*7:lam*8])
    dis5 = hamming_dist(s[lam*8:lam*9], s[lam*9:lam*10])
    quo = (dis+dis2+dis3+dis4+dis5)/lam
    quotients.append([quo, lam])
min_lam = min(quotients)
quotients.sort()
min3_lam = quotients[:6]

In [31]:
n = quotients[5][1]
blocks = [s[i:i+n] for i in range(0, len(s), n)]
new_blocks = []
for i in range(n):
    new_block = b''
    for blck in blocks:
        try:
            new_block += bytes([blck[i]])
        except:
            pass
    new_blocks.append(new_block)

In [32]:
super_key = ""
for new_bl in new_blocks:
    out = try_decrypt_xor(new_bl)
    super_key += out[0]
super_key = str.encode(super_key)
print(super_key)

b'Bigger number, better person'


In [33]:
print(xor_repeating(s, super_key))

b'NARRATOR: Japan is an island by the sea filled with volcanoes and it\'s...\n\nCHORUS: Beautiful!\n\nNARRATOR: In the year -1,000,000,000... Japan might not have been here. In the year -40,000 it was here and you could walk to it. And some people walked to it.\n\nThe year is now -12,000\n\nNARRATOR: Then it got warmer, some icebergs melted, it became an island, and now there\'s lots of trees because it\'s warmer.\n\nThe year is now -10,000\n\nNARRATOR: So, now there\'s people on the island and they\'re basically sort of hanging out in between the mountains eating nuts off trees and using the latest technology. Like stones and bowls.\n\nThe sound of a doorbell chime. The year is now -500.\n\nOUTSIDE WORLD (KOREA): Ding dong.\n\nNARRATOR: It\'s the outside world, and they have technology from the future, like really good metal and crazy rice farms!\nNow, you can make a lot of rice really, really quickly.\nThat means if you own the farm, you own a lot of food, which is something everybod